In [1]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='.\data\ENZYMES', name='ENZYMES')

Extracting .\data\ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


In [3]:
print(dataset)
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)

ENZYMES(600)
600
6
3


In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset

In [5]:
data = dataset[14]
print(data)
print(data.is_undirected())

Data(edge_index=[2, 128], x=[36, 3], y=[1])
True


In [6]:
Data(edge_index=[2, 128], x=[36, 3], y=[1])

Data(x=[2], edge_index=[2], y=[1])

In [3]:
#begin from this section


from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_scatter import scatter_mean
from torch_geometric.datasets import Planetoid
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset
dataset = Planetoid(root='.\data\Cora', name='Cora')########  .\ \
data = dataset[0]

print(data)
print(data.is_undirected())
print(data.train_mask.sum().item())
print(data.val_mask.sum().item())
print(data.test_mask.sum().item())

print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)
#————————————————
#版权声明：本文为CSDN博主「喵木木」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
#原文链接：https://blog.csdn.net/Jenny_oxaza/article/details/107561125
#dataset = TUDataset(root='.\data\ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in loader:
    print(data)
    print(data.num_graphs)
    x = scatter_mean(data.x, data.batch, dim=0)
    print(x.size())
#————————————————
#版权声明：本文为CSDN博主「喵木木」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
#原文链接：https://blog.csdn.net/Jenny_oxaza/article/details/107561125

Using existing file ind.cora.x
Processing...
Done!
/home/zhengfang/anaconda3/envs/new_pytorch/lib/python3.6/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
True
140
500
1000
1
7
1433
DataBatch(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], batch=[2708], ptr=[2])
1
torch.Size([1, 1433])


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
#————————————————
#版权声明：本文为CSDN博主「喵木木」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
#原文链接：https://blog.csdn.net/Jenny_oxaza/article/details/107561125

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))


Accuracy: 0.8000
